In [1]:
import torch
import sys
import iree.turbine.aot as aot

sys.path.append("../clean_clones/Automatic-Speech-Recognition")

from asr.models.quartznet import QuartzNet

/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
model = QuartzNet()
# model.load_state_dict(
#     torch.load(
#         "../models/yolox_nano.pth", map_location=torch.device("cpu"), weights_only=False
#     )["model"],
# )

model = model.eval()
sample_inputs = torch.randn((1, 300, 128))

print(f"{model.forward(sample_inputs)['logits'].shape}")


class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x)["logits"]


model_ = Wrapper(model).cpu().eval()
export_output = aot.export(model_, sample_inputs)
export_output.save_mlir("converted_quartznet.mlir")

torch.Size([1, 150, 28])


In [7]:
sum([p.numel() for p in model.parameters()]) * 4 / 10**6

24.340592